In [1]:
!pip install fastapi uvicorn nest-asyncio pyngrok python-multipart


In [13]:
from fastapi import FastAPI, UploadFile,File
from fastapi.responses import JSONResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import io
from PIL import Image

In [3]:
from transformers import pipeline


In [4]:
from transformers.utils import logging
logging.set_verbosity_error()

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

In [28]:
!ngrok config add-authtoken "paste your ngrok authtoken here"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [25]:
app = FastAPI()

@app.post("/generate-caption", response_class=JSONResponse)
async def generate_caption(image: UploadFile = File(...)):
    raw_image = await image.read()
    pil_image = Image.open(io.BytesIO(raw_image)).convert("RGB")
    caption = pipe(pil_image)
    print(caption)
    caption = caption[0]["generated_text"]
    return JSONResponse(content={"caption": caption})


In [26]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500", "http://localhost:5000"],  # add all your frontend origins here
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [27]:
# Setup for running inside Jupyter/Colab
nest_asyncio.apply()
ngrok.kill()
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run API
uvicorn.run(app, port=8000)

Public URL: NgrokTunnel: "https://10d2670f30e5.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1200]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


[{'generated_text': 'a person sitting on a rock in the water at sunset'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'a woman taking a picture with a camera in a field'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'there is a blue bird sitting on a branch of a tree'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'there is a blue bird sitting on a branch of a tree'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'a person sitting on a rock in the water at sunset'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'there is a tiger that is sitting on a rock in the grass'}]
INFO:     103.179.246.207:0 - "POST /generate-caption HTTP/1.1" 200 OK
[{'generated_text': 'mountains covered in fog and low clouds with houses on the hillside'}]
INFO:     103.179.246.207:0 - 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1200]
